- Set up the Environment

In [1]:
import os
import sys
sys.path.append(os.getcwd())

import time
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.metrics import r2_score

from tkat_loc import TKAT # this is the KAN version with Transformer

tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

from tkat_model_utils import generate_data_w_known_inputs

- Read sample financial data

In [2]:
df = pd.read_parquet('data.parquet')
df = df[(df.index >= pd.Timestamp('2020-01-01')) & (df.index < pd.Timestamp('2023-01-01'))]
# assets = ['BTC', 'ETH', 'ADA', 'XMR', 'EOS', 'MATIC', 'TRX', 'FTM', 'BNB', 'XLM', 'ENJ', 'CHZ', 'BUSD', 'ATOM', 'LINK', 'ETC', 'XRP', 'BCH', 'LTC']
assets = ['BTC']
df = df[[c for c in df.columns if 'quote asset volume' in c and any(asset in c for asset in assets)]]
df.columns = [c.replace(' quote asset volume', '') for c in df.columns]
known_input_df = pd.DataFrame(index=df.index, data=np.array([df.reset_index()['group'].apply(lambda x: (x.hour)).values, df.reset_index()['group'].apply(lambda x: (x.dayofweek)).values]).T, columns = ['hour', 'dayofweek'])
display(df)
display(known_input_df)

,BTC
group,
2020-01-01 00:00:00,3.675857e+06
2020-01-01 01:00:00,6.365953e+06
2020-01-01 02:00:00,4.736719e+06
2020-01-01 03:00:00,5.667367e+06
2020-01-01 04:00:00,3.379094e+06
...,...
2022-12-31 19:00:00,6.704605e+07
2022-12-31 20:00:00,4.344849e+07
2022-12-31 21:00:00,5.992803e+07


,hour,dayofweek
group,,
2020-01-01 00:00:00,0,2
2020-01-01 01:00:00,1,2
2020-01-01 02:00:00,2,2
2020-01-01 03:00:00,3,2
2020-01-01 04:00:00,4,2
...,...,...
2022-12-31 19:00:00,19,5
2022-12-31 20:00:00,20,5
2022-12-31 21:00:00,21,5


- Read last 1000 rows of data for quicker runtime

In [3]:
df = df.tail(1000)
known_input_df = known_input_df.tail(1000)

- Define model parameters

In [4]:
N_MAX_EPOCHS = 100
BATCH_SIZE = 128
early_stopping_callback = lambda : tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=6,
    mode="min",
    restore_best_weights=True,
    start_from_epoch=6,
)
lr_callback = lambda : tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=3,
    mode="min",
    min_delta=0.00001,
    min_lr=0.000025,
    verbose=0,
)
callbacks = lambda : [early_stopping_callback(), lr_callback(), tf.keras.callbacks.TerminateOnNaN()]

In [5]:
num_hidden = 100
num_heads = 4
num_embedding = 1
n_ahead = 4
sequence_length = 5 * n_ahead

- Transform dataframe to correct format for TKAT

In [6]:
X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test = generate_data_w_known_inputs(
    df, 
    known_input_df, 
    sequence_length, 
    n_ahead
    )

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((509, 24, 3), (128, 24, 3), (509, 4), (128, 4))

- One-Hot encoding for classification

In [7]:
y_train = np.random.randint(0, n_ahead, y_train.shape[0])
y_test = np.random.randint(0, n_ahead, y_test.shape[0])

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(max_categories=n_ahead)
y_train = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_test = enc.fit_transform(y_test.reshape(-1,1)).toarray()

In [8]:
num_unknow_features = len(assets)
num_know_features = X_train.shape[2] - num_unknow_features

- Compile TKAT model for classification

In [13]:
from keras.losses import CategoricalCrossentropy
model = TKAT(sequence_length, num_unknow_features, num_know_features, num_embedding, num_hidden, num_heads, n_ahead, use_tkan = True)

# model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy']) # from_logits, allow us to automatically perform softmax activation in the loss

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 24, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_layer_1   │ (None, 24, 1, 3)  │          0 │ input_layer_1[0]… │
│ (EmbeddingLayer)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ past_observed_and_… │ (None, 20, 1, 3)  │          0 │ embedding_layer_… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vsn_past_features   │ (None, 20, 100)   │    103,824 │ past_observed_an… │
│ (VariableSelection… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ future_known        │ (None, 4, 1, 2)   │          0 │ embedding_layer_… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder             │ [(None, 20, 100), │     71,612 │ vsn_past_feature… │
│ (RecurrentLayer)    │ (None, 100),      │            │                   │
│                     │ (None, 100),      │            │                   │
│                     │ (None, 100)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vsn_future_features │ (None, 4, 100)    │     72,614 │ future_known[0][… │
│ (VariableSelection… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder             │ (None, 4, 100)    │     71,612 │ vsn_future_featu… │
│ (RecurrentLayer)    │                   │            │ encoder[0][1],    │
│                     │                   │            │ encoder[0][2],    │
│                     │                   │            │ encoder[0][3]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 24, 100)   │          0 │ encoder[0][0],    │
│ (Concatenate)       │                   │            │ decoder[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gate_16 (Gate)      │ (None, 24, 100)   │          0 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 24, 100)   │          0 │ vsn_past_feature… │
│ (Concatenate)       │                   │            │ vsn_future_featu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_and_norm_16     │ (None, 24, 100)   │        200 │ gate_16[0][0],    │
│ (AddAndNorm)        │                   │            │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ grn_15 (GRN)        │ (None, 24, 100)   │          0 │ add_and_norm_16[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 100)   │    161,300 │ grn_15[0][0],     │
│ (MultiHeadAttentio… │                   │            │ grn_15[0][0],     │
│                     │                   │            │ grn_15[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 2400)      │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_89 (Dense)    │ (None, 4)         │      9,604 │ flatten_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 490,766 (1.87 MB)

 Trainable params: 490,746 (1.87 MB)

 Non-trainable params: 20 (80.00 B)

- Model Training

In [14]:
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30, validation_split=0.2, 
                    callbacks=callbacks(), 
                    shuffle=True, verbose = True)

Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.2434 - loss: 1.6677 - val_accuracy: 0.2647 - val_loss: 1.6477 - learning_rate: 0.0010
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - accuracy: 0.2280 - loss: 1.6141 - val_accuracy: 0.2255 - val_loss: 1.4554 - learning_rate: 0.0010
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step - accuracy: 0.2948 - loss: 1.3879 - val_accuracy: 0.2647 - val_loss: 1.4477 - learning_rate: 0.0010
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.2467 - loss: 1.4106 - val_accuracy: 0.2353 - val_loss: 1.4342 - learning_rate: 0.0010
Epoch 5/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.2911 - loss: 1.3864 - val_accuracy: 0.2451 - val_loss: 1.4891 - learning_rate: 0.0010
Epoch 6/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - accuracy: 0.3287 - loss: 1.3729 - val_accuracy: 0.2549 - val_loss: 1.4558 - learning_rate: 0.0010
Epoch 7/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - accuracy: 0.2912 - loss: 1.3739 - val_accuracy: 0.

- Run predictions

In [ ]:
pred_labels = model.predict(X_test).argmax(axis=1)
true_labels = y_test.argmax(axis=1)

- Run Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_labels))

## Hyperparemeter Tuning

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.model_selection import TimeSeriesSplit
import torch
from sklearn.metrics import log_loss
import optuna
import numpy as np
from sklearn.model_selection import KFold
from torch.utils.data import Subset, DataLoader
from torch.nn.functional import nll_loss

def tkat_objective(trial):

    X, y = X_train, y_train

    # Suggest hyperparameters
    # lr = trial.suggest_categorical('lr', [1e-5, 1e-3, 1e-2])
    num_heads = trial.suggest_categorical('num_heads', [1, 2, 4])
    # dropout_prob = trial.suggest_categorical('dropout_prob', [0.1, 0.3, 0.5])
    hidden_units = trial.suggest_categorical('hidden_units', [64, 128, 256])
    # lstm_layers = trial.suggest_categorical('lstm_layers', [1, 2, 4])
    num_embedding = trial.suggest_categorical('classifier_units', [16, 32, 64])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])

    # Initialize the model with suggested hyperparameters
    model = TKAT(sequence_length, num_unknow_features, num_know_features, num_embedding, hidden_units, num_heads, n_ahead, use_tkan = True)
    
    model.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    
    # Time series split
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = []

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):

        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        history = model.fit(X_train_fold, y_train_fold, batch_size=BATCH_SIZE, epochs=30, 
                    callbacks=callbacks(), 
                    shuffle=False, verbose = True)

        # Validate the model

        pred_labels = model.predict(X_val_fold).argmax(axis=1)
        true_labels = y_val_fold.argmax(axis=1)

        # val_predictions = trainer.predict(model, val_loader)
        # val_predictions = torch.cat([x for x in val_predictions], dim=0).numpy()
        
        val_loss = nll_loss(pred_labels, true_labels)
        cv_scores.append(val_loss)

    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(tkat_objective, n_trials=5)

In [ ]:
# Print best hyperparameters
print("Best hyperparameters:", study.best_params)

In [ ]:
from keras.losses import CategoricalCrossentropy
final_model = TKAT(
    sequence_length, 
    num_unknow_features, 
    num_know_features, 
    study.best_params['num_embedding'], 
    study.best_params['hidden_units'], 
    study.best_params['num_heads'], 
    n_ahead, 
    use_tkan = True)

# model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy']) # from_logits, allow us to automatically perform softmax activation in the loss

model.summary()

In [ ]:
history = final_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30, validation_split=0.2, 
                    callbacks=callbacks(), 
                    shuffle=True, verbose = True)

### Cross - Validation

In [ ]:
from tkat_model_utils import cross_validate_model
cv_results = cross_validate_model(
    X_test, 
    num_splits=5, 
    model=final_model, 
    num_epochs=10, 
    num_classes=n_ahead)

In [ ]:
print(cv_results)

### MC Dropout Predictions

In [ ]:
from tkat_model_utils import mc_dropout_predictions 
# Perform MC Dropout predictions
mc_predictions = mc_dropout_predictions(final_model, X_test)

100%|██████████| 100/100 [12:02<00:00,  7.22s/it]


In [ ]:
mean_predictions = mc_predictions.mean(axis=0).squeeze()
std_predictions = mc_predictions.std(axis=0).squeeze()
predicted_labels = np.argmax(mean_predictions, axis=1)

### Results exporting

- Export the reuslts

In [ ]:
import pandas as pd
# Save test predictions to a CSV
test_df = pd.DataFrame({
    'Prediction': predicted_labels,
    'Probability_0': [p[0] for p in mean_predictions],
    'Probability_1': [p[1] for p in mean_predictions],
    'Probability_2': [p[2] for p in mean_predictions],  # Adjust based on num_classes
    'Probability_3': [p[3] for p in mean_predictions],
    'Uncertainty_0': [u[0] for u in std_predictions],
    'Uncertainty_1': [u[1] for u in std_predictions],
    'Uncertainty_2': [u[2] for u in std_predictions],
    'Uncertainty_3': [u[3] for u in std_predictions]
})

test_df.to_csv('tkat_predictions.csv', index=False)

- Pytorch-Lightning: Save the model

In [ ]:
from torch import save
save(model.state_dict(), 'tkat_classifier.pth')